In [1]:
import cv2
from darkflow.net.build import TFNet
import numpy as np
import time

//anaconda/envs/py35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
option = {
    'model': 'cfg/yolo.cfg',
    'load': 'bin/yolo.weights',
    'threshold': 0.35,
    'gpu': 0.0
}

tfnet = TFNet(option)

Parsing ./cfg/yolo.cfg
Parsing cfg/yolo.cfg
Loading bin/yolo.weights ...
Successfully identified 203934260 bytes
Finished in 0.26988792419433594s
Model has a coco model name, loading coco labels.

Building net ...
Source | Train? | Layer description                | Output size
-------+--------+----------------------------------+---------------
       |        | input                            | (?, 608, 608, 3)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 608, 608, 32)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 304, 304, 32)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 304, 304, 64)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 152, 152, 64)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 152, 152, 128)
 Load  |  Yep!  | conv 1x1p0_1  +bnorm  leaky      | (?, 152, 152, 64)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 152, 152, 128)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 76, 76, 128)
 Load  |  Yep!

In [3]:
#capture = cv2.VideoCapture('/media/cfchen/956df7bc-562e-4f24-8339-fd0b67f98888/Downloaded/VideosHPB/IMAG0011.mp4')
capture = cv2.VideoCapture('../VideosHPB/IMAG0011.mp4')
colors = [tuple(255 * np.random.rand(3)) for i in range(5)]  #range(100)

In [4]:
for color in colors:
    print(color)

(252.14365638822247, 93.44724034046668, 6.837202584994625)
(136.98495290698395, 62.8169061586848, 174.08569267662614)
(96.50646335006104, 149.16109910991477, 244.36384583309695)
(54.370934070516086, 185.1158363729087, 250.99700249874022)
(211.7351446782044, 237.41045777467963, 196.10835026004528)


In [5]:
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi',fourcc, 30.0, (1920,1080))

## Test

In [6]:
ret, frame = capture.read()

In [7]:
print(frame.shape)
type(frame)

(1080, 1920, 3)


numpy.ndarray

In [8]:
results = tfnet.return_predict(frame)
type(results)

list

## Run the Video

In [9]:
count = 0
while(True):
    stime = time.time()
    ret, frame = capture.read()

    if ret == False:
        break;
    else:
        count = count + 1

    results = tfnet.return_predict(frame)

    if ret:
        for color, result in zip(colors, results):
            tl = (result['topleft']['x'], result['topleft']['y'])
            br = (result['bottomright']['x'], result['bottomright']['y'])
            label = result['label']
            frame = cv2.rectangle(frame, tl, br, color, 7)
            frame = cv2.putText(frame, label, tl, cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 0), 2)

        out.write(frame)
        cv2.imshow('frame', frame)
        print('FPS {:.1f}'.format(1 / (time.time() - stime)))  # Frame per seconds
        if (cv2.waitKey(1) & 0xFF == ord('q')) or (ret == False):
            break

print('++++++++ Video End ++++++++')
  
capture.release()
out.release()
cv2.destroyAllWindows()

FPS 0.2
FPS 0.3
FPS 0.2
FPS 0.3
FPS 0.3
FPS 0.3
FPS 0.2
FPS 0.3
++++++++ Video End ++++++++
